In [ ]:
%matplotlib inline
from marketdata import *
from model import *
from analytics import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from datetime import datetime

import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

# Load data 

Try clean up markets with 100% close out values

In [ ]:
list_of_markets=get_market_list()

In [ ]:
df=pd.DataFrame()
for m in list_of_markets:
    try:
        df[m]=get_timeseries(m)
    except:
        print m

In [ ]:
def remove_inf(pnl):
    return pnl.replace(np.inf,0).replace(-np.inf,0)

In [ ]:
mkts=df.T[df.count().sort_values()!=0].index

In [ ]:
pos=tsmom_daily(df[mkts],30)
pnl=calc_pnl(pos,df[mkts])
scaling_factor=calc_scaling_factor(pnl)
portfolio= remove_inf(pnl).mean(axis=1).dropna()*scaling_factor

In [ ]:
calc_Sharpe(pnl).dropna().sort_values().plot(kind='bar')

In [ ]:
portfolio.cumsum().plot()

In [ ]:

Sigma_all=pnl.cov().dropna(how='all',axis=1).dropna(how='all')
ind=pnl.dropna(how='all').index
SF=[]
for row in pnl.dropna(how='all').iterrows():
    no_mkts=row[1].dropna().count()
    w=np.array([1/float(no_mkts)]*no_mkts)
    mkt_list=row[1].dropna().index
    Sig = Sigma_all[mkt_list].T[mkt_list]
    vol_ach=np.sqrt(np.dot(np.dot(w.T,Sig.as_matrix()),w))*16
    SF.append(vol_target/vol_ach)
